# Assignment II - Object detection using Deep Learning

### Including the Appropriate Libraries

In [1]:
from pathlib import Path
import argparse
import logging
import json
import shutil
import numpy as np
import cv2
from PIL import Image
from tqdm import tqdm
import yaml
import requests
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
import torchvision
# yolo_imports.py
# Imports commonly required for YOLO-based object detection workflows (PyTorch/Ultralytics/yolov5)
import matplotlib.pyplot as plt
import torchvision.transforms as T



### 1. Use Case Definition: 

- Select a real-world use case for object detection. 
- This could be anything from pedestrian detection for autonomous vehicles to detecting 
specific objects in medical images. 
- Clearly define the problem you intend to solve and explain why object detection is necessary for this use case. 


### 2. Data Collection: 
- Gather a dataset of images relevant to your use case. 
- Ensure that the dataset includes annotated bounding boxes around the objects of interest.
- You may need to annotate the images yourself or find pre-annotated datasets.

In [2]:
torch.cuda.is_available()

True

In [4]:
!python yolov7/train.py \
  --img 640 \
  --batch 8 \
  --epochs 10 \
  --data images/hardhat.yv7pytorch/data.yaml \
  --weights yolov7/yolov7.pt \
  --cache \
  --device 0


YOLOR 🚀 v0.1-128-ga207844 torch 2.5.1 CUDA:0 (NVIDIA GeForce RTX 3070 Laptop GPU, 7973.6875MB)

Namespace(weights='yolov7/yolov7.pt', cfg='', data='images/hardhat.yv7pytorch/data.yaml', hyp='./yolov7/data/hyp.scratch.p5.yaml', epochs=10, batch_size=8, img_size=[640, 640], rect=False, resume=False, nosave=False, notest=False, noautoanchor=False, evolve=False, bucket='', cache_images=True, image_weights=False, device='0', multi_scale=False, single_cls=False, adam=False, sync_bn=False, local_rank=-1, workers=8, project='runs/train', entity=None, name='exp', exist_ok=False, quad=False, linear_lr=False, label_smoothing=0.0, upload_dataset=False, bbox_interval=-1, save_period=-1, artifact_alias='latest', freeze=[0], v5_metric=False, world_size=1, global_rank=-1, save_dir='runs/train/exp4', total_batch_size=8)
tensorboard: Start with 'tensorboard --logdir runs/train', view at http://localhost:6006/
hyperparameters: lr0=0.01, lrf=0.1, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, war

In [10]:
!python yolov7/detect.py --weights runs/train/exp4/weights/best.pt \\
--weights runs/train/exp4/weights/best.pt \
--source images/hardhat.yv7pytorch/test/images/

Namespace(weights=['runs/train/exp4/weights/best.pt'], source='images/hardhat.yv7pytorch/test/images/', img_size=640, conf_thres=0.25, iou_thres=0.45, device='', view_img=False, save_txt=False, save_conf=False, nosave=False, classes=None, agnostic_nms=False, augment=False, update=False, project='runs/detect', name='exp', exist_ok=False, no_trace=False)
YOLOR 🚀 v0.1-128-ga207844 torch 2.5.1 CUDA:0 (NVIDIA GeForce RTX 3070 Laptop GPU, 7973.6875MB)

/home/sebas/Desktop/ie_dev/y4.1/advancedAI/assignments/hw2-objectDetection/yolov7/models/experimental.py:252: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This lim

In [18]:
!python yolov7/detect.py \
  --weights runs/train/exp4/weights/best.pt \
  --source images/hardhat.yv7pytorch/test/images \
  --agnostic-nms


Namespace(weights=['runs/train/exp4/weights/best.pt'], source='images/hardhat.yv7pytorch/test/images', img_size=640, conf_thres=0.25, iou_thres=0.45, device='', view_img=False, save_txt=False, save_conf=False, nosave=False, classes=None, agnostic_nms=True, augment=False, update=False, project='runs/detect', name='exp', exist_ok=False, no_trace=False)
YOLOR 🚀 v0.1-128-ga207844 torch 2.5.1 CUDA:0 (NVIDIA GeForce RTX 3070 Laptop GPU, 7973.6875MB)

/home/sebas/Desktop/ie_dev/y4.1/advancedAI/assignments/hw2-objectDetection/yolov7/models/experimental.py:252: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limit

In [22]:
!python yolov7/test.py \
  --data images/hardhat.yv7pytorch/data.yaml \
  --weights runs/train/exp4/weights/best.pt \
  --img 640 --conf-thres 0.001 --iou-thres 0.65


Namespace(weights=['runs/train/exp4/weights/best.pt'], data='images/hardhat.yv7pytorch/data.yaml', batch_size=32, img_size=640, conf_thres=0.001, iou_thres=0.65, task='val', device='', single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project='runs/test', name='exp', exist_ok=False, no_trace=False, v5_metric=False)
YOLOR 🚀 v0.1-128-ga207844 torch 2.5.1 CUDA:0 (NVIDIA GeForce RTX 3070 Laptop GPU, 7973.6875MB)

/home/sebas/Desktop/ie_dev/y4.1/advancedAI/assignments/hw2-objectDetection/yolov7/models/experimental.py:252: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will b

### 3. Data Preprocessing: 

- Preprocess the dataset by resizing images, normalizing 
pixel values, and splitting it into training, validation, and test sets. 

- Ensure that the data is properly formatted for YOLO training. 

### 4. YOLO fine-tuning: 

- Leverage the YOLO algorithm using one of the solutions 
already implemented in GitHub. 

- Fine-tune the YOLO model using the training dataset for optimal object detection performance. 

### 5. Model Evaluation: 

- Evaluate the trained YOLO model on the test dataset. 
- Calculate object detection metrics such as mean average precision (mAP) and IoU (Intersection over Union) to assess its performance. 

### 6. Results and Discussion: 

- Present the results of your object detection task and 
discuss the practical implications of your use case. 

- Reflect on the challenges and limitations encountered during the implementation and evaluation process.